# knn 最近邻域算法

    将测试图片，与训练图片进行比较，如果存在k张与测试图片最近的训练图片，选取出现的概率最高的训练图片，则概率最高的图片的标签值记录下来，就是最终检测出来的结果
    
步骤：
1. 准备数据
2. 训练数据，得到：像素间距离==>累加求和==>图片距离
3. 找到k个最接近的图片
4. 使用标签值解析图片
5. 将标签值转成具体的数字
6. 概率检测统计

In [1]:
import tensorflow as tf
import numpy as np
import random             # 生成随机数组，用来测试图片
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist", one_hot=True)    # 参数1：文件路径，参数2：one-hot编码


# 属性设置
trainNum = 55000     # 总共有55000张训练图片
testNum = 10000      # 总共有10000张测试图片
trainSize = 500      # 随机选出训练图片的数量
testSize = 5         # 随机选出测试图片的数量


# 随机获取训练图片(样本)和测试图片(样本)，以及相应的标签值
trainIndex = np.random.choice(trainNum, trainSize, replace=False)   # 范围0~trainNum之间，随机选择trainSize个数字，不可重复。在0~55k中，随机选取500个数字
testIndex = np.random.choice(testNum, testSize, replace=False)

trainData = mnist.train.images[trainIndex]        # 随机选出训练图片，(500, 784)，500张图片，784=图片高宽28*28(图片上所有的像素点)
trainLabel = mnist.train.labels[trainIndex]       # 训练图片的标签值，(500, 10)，500张图片，10个目标值(one-hot)

testData = mnist.train.images[testIndex]          # 随机选出测试图片，(5, 784)
testLabel = mnist.train.labels[testIndex]         # 测试图片的标签值，(5, 10)

# 找到k个与测试图片相近的图片，并且统计这些图片中，一个类别出现次数最多的图片，则这个类别就是最终的数据
k = 4

# 准备tf的输入数据（train+test）
trainDataInput = tf.placeholder(shape=[None, 784], dtype=tf.float32)    # 加载输入数据，每一行的784列表示一个张完整的图片，None表示图片的张数，由具体的特征值类型决定
trainLabelInput = tf.placeholder(shape=[None, 10], dtype=tf.float32)    # 每一张图片的标签值，都是一个10列one-hot编码

testDataInput = tf.placeholder(shape=[None, 784], dtype=tf.float32)
testLabelInput = tf.placeholder(shape=[None, 10], dtype=tf.float32)


# knn distance：
# 1.【训练图片】增加一维：(5,784) ==> (5,1,784)，5张图片每1张图片784个像素点。
# 2. 计算5张【训练图片】中的每1张图片和500张【测试图片】之间，784个像素点的距离差。(5,500,784) 平面数：【测试图片数据】。行数：【训练图片数据】。列数：【两张图片距离之差】
# 3. 降维累加：计算【5张测试图】与【500张训练图】之间的距离差值。(5,500) 
f1 = tf.expand_dims(testDataInput,1)    # 维度转换
f2 = tf.subtract(trainDataInput,f1)     # 计算距离差
f3 = tf.reduce_sum(tf.abs(f2),reduction_indices=2)    # 把784像素点的差值(降维)累加。参数2：在哪一个维度进行数据累加

# 例：(1,100) 表示，第2张测试图片，与第99张训练图片之间的所有像素的距离之差

# 如何根据计算出来的距离，找到knn中k个最近的图片。
# 在5张【测试图片】中，使用每1张训练图片在【500张训练图】中找出4个相似的图片
f4 = tf.negative(f3)    # 取反，130==> -130
f5,f6 = tf.nn.top_k(f4, k=4)    # 选取f4中，最大的4个数。f3最小的4个数，即最接近测试图片的4个数。f5：距离差(4个值)，f6：4张最近的训练图片的下标

# 通过k个相似图的label属性，解析k个相似的图。
f7 = tf.gather(trainLabelInput, f6)      # 获取所有标签

# 数字获取
f8 = tf.reduce_sum(f7, reduction_indices=1)    # 竖直方向，列累加
f9 = tf.argmax(f8, dimension=1)    # 返回f8中，每行最大值的下标，即5张测试图片经过训练得出的5个标签值

# 将f9与testLabel比较，如果相同，则检测成功率为100%。一半相同，则50%


with tf.Session() as sess:
    f1_res = sess.run(f1, feed_dict={testDataInput:testData})    # (5,1,784)，注:testData = testData[0:5]左闭右开
    f2_res = sess.run(f2, feed_dict={trainDataInput:trainData, testDataInput:testData})    # (5,500,784)，注:一个op用了n个占位符，就得传n个占位符
    f3_res = sess.run(f3, feed_dict={trainDataInput:trainData, testDataInput:testData})    # (5,500)
    f4_res = sess.run(f4, feed_dict={trainDataInput:trainData, testDataInput:testData})    # (5,500)

    print("f3_res[0,0]:", f3_res[0,0])  # 158.20392
    print("f4_res[0,0]:", f4_res[0,0])  # -158.20392
    
    f5_res, f6_res = sess.run((f5, f6), feed_dict={trainDataInput:trainData, testDataInput:testData})     # f5_res(5,4);f6_res(5,4)，5张测试图片的每1张测试图片对应4张训练图片
    # f5_res是每1张测试图片到4张训练图片的距离，f6_res是每1张测试图片对应的训练图片的下标index
#     print("f5_res[0,0]:", f5_res[0,0])
#     print("f6_res[0,0]:", f6_res[0,0])
#     print("f5_res[0]:", f3_res[0])
#     print("f6_res[0]:", f3_res[0])
    
    f7_res = sess.run(f7, feed_dict={trainDataInput:trainData, testDataInput:testData, trainLabelInput:trainLabel})    # (5,4,10)
    print("f7_res",f7_res)
    
    f8_res, f9_res = sess.run((f8, f9), feed_dict={trainDataInput:trainData, testDataInput:testData, trainLabelInput:trainLabel})    # f8_res(5,10), f9_res(5,)
    print("f8_res",f8_res)
    print("f9_res",f9_res)
    
    label_res = np.argmax(testLabel, axis=1)
    print(label_res)
    

# 检测概率
j = 0
for i in range(5): # 在5张图片的标签值中遍历
    if label_res[i] == f9_res[i]:
        j = j+1
print("准确率：", j*100/5)    # 小数转成百分数，5组数据

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use the `axis` argument instead
f3_res[0,0]: 92.807846
f4_res[0,0]: -92.807846
f7_res [[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0.

# CNN

In [2]:
"""
卷积模型（确定网络结构和参数）：
两层卷积池化和一层输出层

第一层
    卷积：32个filter、大小5*5、strides=1、padding="SAME"
    激活：Relu
    池化：大小 2x2、strides = 2
第二层
    卷积：64个filter、大小5*5、strides=1、padding="SAME"
    激活：Relu
    池化：大小 2x2、strides = 2
全连接层

重点：计算每层数据的变化

梯度爆炸：
1. 调整参数w,b的值
2. 使用tf.train.AdamOptimizer()
"""

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


# 定义专门初始化权重和偏置的两个函数（因为两个参数的形状不同，所以要把权重和偏置分开）
def weight_initialize(shape):
    weight = tf.Variable(tf.random_normal(shape=shape, mean=0.0, stddev=0.1), name="weight")
    return weight


def bias_initialize(shape):
    bias = tf.Variable(tf.random_normal(shape=shape, mean=0.0, stddev=0.1), name="bias")
    return bias


def cnn_model():
    """自定义卷积模型"""
    # 1. 定义特征值和目标值的占位符，便于卷积计算:x[None,784], y[None,10]
    with tf.variable_scope("data"):
        x = tf.placeholder(tf.float32, [None, 784], name="feature")
        y_true = tf.placeholder(tf.float32, [None, 10], name="y_true")

    # 2. 第一层：
    with tf.variable_scope("layer_1"):
        # 2.1 卷积层：32个filter、大小5*5、strides=1、padding="SAME"

        # 初始化权重[5,5,1,32] 和 偏置[32]
        conv1_weight = weight_initialize([5, 5, 1, 32])
        conv1_bias = bias_initialize([32])

        # 特征形状变成4维，用于卷积运算。（重点：None就是-1）
        x_reshape = tf.reshape(x, [-1, 28, 28, 1])

        # 卷积运算: [-1,28,28,1] --> [-1,28,28,32]
        conv1_x = tf.nn.conv2d(input=x_reshape,
                               filter=conv1_weight,
                               strides=[1, 1, 1, 1],
                               padding="SAME",
                               name="conv1") + conv1_bias
        # 2.2 激活层运算
        relu1_x = tf.nn.relu(conv1_x, name="relu1")

        # 2.3 池化层: 大小 2x2、strides=2、padding="SAME"
        # [-1,28,28,32] --> [-1,14,14,32]
        pool1_x = tf.nn.max_pool(value=relu1_x,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding="SAME",
                                 name="pool1")

    # 3. 第二层
    with tf.variable_scope("layer_2"):
        # 3.1 卷积层：64个filter、大小5*5、strides=1、padding="SAME"

        # 初始化权重[5,5,32,64] 和 偏置[64]
        conv2_weight = weight_initialize([5, 5, 32, 64])
        conv2_bias = bias_initialize([64])

        # 卷积运算：[-1,14,14,32] --> [-1,14,14,64]
        conv2_x = tf.nn.conv2d(input=pool1_x,
                               filter=conv2_weight,
                               strides=[1, 1, 1, 1],
                               padding="SAME",
                               name="conv1") + conv2_bias

        # 3.2 激活层运算
        relu2_x = tf.nn.relu(conv2_x, name="relu2")

        # 2.3 池化层: 大小 2x2、strides=2、padding="SAME"
        # [-1,14,14,64] --> [-1,7,7,64]
        pool2_x = tf.nn.max_pool(value=relu2_x,
                                 ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1],
                                 padding="SAME",
                                 name="pool2")

    # 4. 全连接层
    with tf.variable_scope("last_layer"):
        """10个输出"""
        # 初始化权重[7*7*64,10] 和 偏置[10]
        fc_weight = weight_initialize([7 * 7 * 64, 10])
        fc_bias = bias_initialize([10])

        # 4维转换2维
        pool2_x_reshape = tf.reshape(pool2_x, [-1, 7 * 7 * 64])

        # 全连接层矩阵运算
        y_predict = tf.matmul(pool2_x_reshape, fc_weight) + fc_bias

    return x, y_true, y_predict


def cnn_mnist():
    """卷积网络识别训练"""

    # 1. 获取数据
    mnist = input_data.read_data_sets("./mnist", one_hot=True)

    # 2. 建立卷积网络模型：（说明）
    x, y_true, y_predict = cnn_model()

    # 3. 根据输出的结果计算其softmax，并与真实值进行交叉熵损失计算
    with tf.variable_scope("softmax_crossentropy"):
        # 先进行网络输出值的softmax概率计算，再计算每个样本的损失
        all_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict, name="compute_loss")

        # 求出平均损失
        loss = tf.reduce_mean(all_loss)

    # 4. 梯度下降优化
    with tf.variable_scope("GD"):
        train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss=loss)
        # 使用adam优化器：
        # train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss=loss)

    # 5. 计算准确率
    with tf.variable_scope("accuracy"):
        equal_list = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_predict, 1))
        accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))


        # 0. 开启会话进行训练（train_op）
        with tf.Session() as sess:
            # 初始化变量OP
            sess.run(tf.global_variables_initializer())

            # 循环训练
            for i in range(100):  # 步数=100
                # 每批次给50个样本
                x_mnist, y_mnist = mnist.train.next_batch(50)
                # run的feed_dict机制，指定给占位符变量传数据。只要运行的变量中含有占位符，都要进行feed_dict
                _, loss_run, accuracy_run= sess.run([train_op, loss, accuracy],
                                                                 feed_dict={x: x_mnist, y_true: y_mnist})

                print("第【%d】步，50个预测图片误差为【%f】，准确率为【%f】" % (i, loss_run, accuracy_run))

    return None


if __name__ == '__main__':
    cnn_mnist()

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

第【0】步，50个预测图片误差为【4.445383】，准确率为【0.060000】
第【1】步，50个预测图片误差为【6.730469】，准确率为【0.120000】
第【2】步，50个预测图片误差为【3.389718】，准确率为【0.080000】
第【3】步，50个预测图片误差为【2.391407】，准确率为【0.080000】
第【4】步，50个预测图片误差为【2.335293】，准确率为【0.100000】
第【5】步，50个预测图片误差为【2.305876】，准确率为【0.040000】
第【6】步，50个预测图片误差为【2.284229】，准确率为【0.140000】
第【7】步，50个预测图片误差为【2.213932】，准确率为【0.160000】
第【8】步，50个预测图片误差为【2.153918】，准确率为【0.220000】
第【9】步，50个预测图片误差为【2.136751】，准确率为【0.280000】
第【10】步，50个预测图片误差为【2.133242】，准确率为【0.260000】
第【11】步，50个预测图片误差为【2.001035】，准确率为【0.220000】
第【12】步，50个预测图片误差为【2.160360】，准确率为【0.200000】
第【13】步，50个预测图片误差为【1.807023】，准确率为【0.520000】
第【14】步，50个预测图片误差为【1.926786】，准确率为【0